Verify that the implementation of the qDRIFT subroutine is correct. 
Compute error statistics for the qDRIFT subroutine.

In [2]:
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
# Adjust the path to the directory containing 'scripts/algo'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts')))
from algo.qft_qpe_qdrift import *
from utils.scalable_numerical_tests import *

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define parameters
times = [10, 1, 0.1, 0.01, 0.001]
for time in times:
    num_qubits = 3
    num_terms = 100
    theoretical_errors = []
    unitary_errors = []
    unitary_errors_inf = []
    diamond_distances = []
    nums_samples = np.arange(500, 2000, 100).tolist() + np.arange(2000, 10000, 1000).tolist()
    # Set seaborn style
    # Generate a random Hamiltonian
    hamiltonian = generate_random_hamiltonian(num_qubits, num_terms)
    print(f"Hamiltonian: {hamiltonian}")

    # Compute exact unitary evolution
    U_exact = exact_unitary_evolution(hamiltonian, time)

    for num_samples in nums_samples:
        # Compute qDRIFT approximation
        sampled_unitaries, labels = qdrift_sample(hamiltonian, time, num_samples)
        # multiply the sampled unitaries in the order they were sampled
        U_qdrift = sampled_unitaries[0]
        for unitary in sampled_unitaries[1:]:
            U_qdrift = U_qdrift @ unitary
        U_qdrift = Operator(U_qdrift)
        # Compute errors
        error = unitary_error_2_norm(U_exact, U_qdrift)
        error_2 = unitary_error_inf_norm(U_exact, U_qdrift)
        diamond_dist = compute_diamond_distance(U_exact, U_qdrift)
        lam = sum(abs(term[0]) for term in hamiltonian)
        theoretical_error = estimate_theoretical_qdrift_errror(num_samples=num_samples, lam=lam, time=time)
        # print(f"Theoretical Error\t\tError\t\tDiamond Distance")
        # print(f"   {theoretical_error}\t    {error} \t    {diamond_dist}")
        theoretical_errors.append(2 * theoretical_error)
        unitary_errors.append(error)
        diamond_distances.append(diamond_dist)
        unitary_errors_inf.append(error_2)

    # graph error vs num sampleswith plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=nums_samples, y=theoretical_errors, mode='lines', name='Theoretical Error'))
    fig.add_trace(go.Scatter(x=nums_samples, y=unitary_errors, mode='lines', name='Unitary Error 2-norm'))
    fig.add_trace(go.Scatter(x=nums_samples, y=unitary_errors_inf, mode='lines', name='Unitary Error inf-norm'))
    fig.add_trace(go.Scatter(x=nums_samples, y=diamond_distances, mode='lines', name='Diamond Distance'))
    fig.update_layout(title=f'Error vs Number of Samples for time={time}', xaxis_title='Number of Samples', yaxis_title='Error')
    fig.show()
    

Hamiltonian: [(0.9465251465472901, Pauli('IIX')), (0.3791426926292165, Pauli('ZXX')), (0.4411056562194777, Pauli('ZXZ')), (0.8758196256212966, Pauli('IXI')), (0.2061392372154054, Pauli('XXZ')), (0.9819217460885482, Pauli('ZZX')), (0.9339937521647721, Pauli('IXI')), (0.42382896024093764, Pauli('IXI')), (0.9587116510843057, Pauli('ZXX')), (0.2649537184973463, Pauli('IXX')), (0.10134585493101222, Pauli('IXZ')), (0.1875341955952028, Pauli('IIX')), (0.5156755914613109, Pauli('IXZ')), (0.8724433616138049, Pauli('ZIZ')), (0.16857809794744472, Pauli('XII')), (0.025377587520212685, Pauli('XZZ')), (0.26734072962912003, Pauli('ZZI')), (0.0928667278016122, Pauli('ZZI')), (0.8152981047963529, Pauli('IZZ')), (0.7824554743726164, Pauli('ZZZ')), (0.20695926828608358, Pauli('ZZI')), (0.9506719251303654, Pauli('IXZ')), (0.09327104328964575, Pauli('IZX')), (0.9337773010260605, Pauli('IXI')), (0.6039000311943997, Pauli('IZZ')), (0.11651258610295634, Pauli('ZZI')), (0.48605603670726805, Pauli('XXZ')), (0.2

Hamiltonian: [(0.16071882225523992, Pauli('XII')), (0.2399538046459363, Pauli('ZIZ')), (0.8233207113029222, Pauli('ZIX')), (0.6842691640523737, Pauli('ZII')), (0.9098793592120075, Pauli('XXI')), (0.08374224063206315, Pauli('XXX')), (0.08189085652510897, Pauli('IIZ')), (0.9698019746778428, Pauli('ZXZ')), (0.3852437075327365, Pauli('XIZ')), (0.40769392212782796, Pauli('ZXX')), (0.04186794849193587, Pauli('XXZ')), (0.37969662487749056, Pauli('IXZ')), (0.09490421525133519, Pauli('ZIZ')), (0.6447683841343456, Pauli('IIX')), (0.4119025480382078, Pauli('IXI')), (0.8260820283298643, Pauli('ZIZ')), (0.5599916071281082, Pauli('ZZX')), (0.10818525180419314, Pauli('ZII')), (0.19724724637601632, Pauli('XIX')), (0.7373805373659424, Pauli('ZXI')), (0.9769456045371838, Pauli('ZXI')), (0.7135471121816601, Pauli('ZZI')), (0.3870764132797413, Pauli('XIZ')), (0.7815188469793777, Pauli('IIZ')), (0.5143780787404049, Pauli('XZZ')), (0.9522020847665692, Pauli('ZXI')), (0.9173534868593151, Pauli('XIZ')), (0.54

KeyboardInterrupt: 